# IBA Probe for East West Traffic

In this tutorial we will demonstrate the "East West Traffic" probe that is used to messure the amount of east/west traffic in your data center fabric.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [3]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our east/west traffic probe

In [4]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [5]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [6]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the east/west traffic probe from the library

In [7]:
from aospy.ibaprobelib.east_west_traffic import eastwest_traffic_probe

Next let's create the east/west traffic probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [8]:
?eastwest_traffic_probe

The following paramaters are required: 
  
`label` -  Descriptive name for the probe  
`average_period` - period over which to average input counter samples  
`history_sample_count` - number of samples of history to maintain  

These values define the sample period which traffic is averaged and the number of samples to keep in history for the probe.

So for the case of reuse, we will define these variables here:

In [9]:
probe_label = 'DC1-East-West'
average_period = 30
history_sample_count = 5000

Now let's create the body data from these variables:

In [10]:
probe_body = eastwest_traffic_probe(label=probe_label, average_period=average_period, history_sample_count=history_sample_count)

If you'd like to dump the contents of the probe_body, you can use the following:

In [ ]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [11]:
probes_url = bp.url + "/probes"

In [12]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [13]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [14]:
resp.ok

True

Now we have our IBA Probe ID:

In [15]:
probe_id = resp.json()['id']

print probe_id

5cae4778-e4e8-4eed-bc0f-c33c0c381967


And now we can see the IBA active probe in the UI:

<img src="media/east_west1.png">

And then select the probe on the UI to get the details:

<img src="media/east_west2.png">

# Delete the IBA East West Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [16]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True